In [12]:
import requests, xmltodict, pandas as pd
from pandas import json_normalize
import re
from pathlib import Path
from zeep import Client, helpers
from utils import safe_serialize, listar_ops

In [2]:
# Si es notebook, mejor usar:
BASE_DIR = Path.cwd()

# Ir un nivel arriba (de notebooks → raíz del repo)
ROOT = BASE_DIR.parent

# Carpeta data dentro del repo
DATA_DIR = ROOT / "data"
DATA_DIR.mkdir(exist_ok=True)

In [3]:
listar_ops("https://opendata.camara.cl/camaradiputados/WServices/WSDiputado.asmx?WSDL")

retornarDiputados()
retornarDiputado(prmDiputadoId)
retornarDiputadosXPeriodo(prmPeriodoID)
retornarDiputadosPeriodoActual()
retornarDiputados()
retornarDiputado(prmDiputadoId)
retornarDiputadosXPeriodo(prmPeriodoID)
retornarDiputadosPeriodoActual()
retornarDiputados()
retornarDiputado(prmDiputadoId)
retornarDiputadosXPeriodo(prmPeriodoID)
retornarDiputadosPeriodoActual()
retornarDiputados()
retornarDiputado(prmDiputadoId)
retornarDiputadosXPeriodo(prmPeriodoID)
retornarDiputadosPeriodoActual()


In [8]:
periodos = ['2018-2022'] # <- Elegir períodos

In [6]:
def estandarizar_df_diputados_x_periodo(client: Client, period_id: int, df_periodos: pd.DataFrame) -> pd.DataFrame:
    """
      - Mantiene sub-tramos de militancia dentro del período legislativo (cambios de partido).
      - Asigna periodo_id y Periodo por TRASLAPE con el período oficial (df_periodos).
      - Calcula militancias_anteriores (inicio < inicio_oficial_período).
      - Calcula edad al inicio oficial del período (o fallback 11-03).
      - Drop/rename y orden final de columnas para CSV.
    """

    # ===== 0) Normaliza df_periodos y saca la fila del período solicitado =====
    dfp = df_periodos.reset_index().copy()
    # Detecta columnas
    # ID
    cand_id = [c for c in dfp.columns if c in ["Id", "ID", "id", "periodo_id", "Id_periodo"]]
    if not cand_id:
        raise ValueError(f"df_periodos no tiene columna ID reconocible. Columnas: {list(dfp.columns)}")
    id_col = cand_id[0]
    # Label
    cand_label = [c for c in dfp.columns if c in ["Periodo", "Nombre", "NombrePeriodo", "periodo", "nombre"]]
    if not cand_label:
        raise ValueError(f"df_periodos no tiene columna de nombre/etiqueta de período. Columnas: {list(dfp.columns)}")
    label_col = cand_label[0]
    # Fechas (pueden o no existir)
    # Si faltan, derivamos (YYYY-YYYY) -> [YYYY-03-11, YYYY2-03-10]
    if "FechaInicio" in dfp.columns:
        dfp["FechaInicio"] = pd.to_datetime(dfp["FechaInicio"], errors="coerce")
    else:
        dfp["FechaInicio"] = pd.NaT
    if "FechaTermino" in dfp.columns:
        dfp["FechaTermino"] = pd.to_datetime(dfp["FechaTermino"], errors="coerce")
    else:
        dfp["FechaTermino"] = pd.NaT

    def _derive_dates_from_label(lbl):
        try:
            a, b = str(lbl).split("-")
            y1, y2 = int(a), int(b)
            ini = pd.Timestamp(f"{y1}-03-11")
            fin = pd.Timestamp(f"{y2}-03-10")
            return ini, fin
        except Exception:
            return pd.NaT, pd.NaT

    # Completa fechas faltantes desde el label
    mask_need_ini = dfp["FechaInicio"].isna()
    mask_need_fin = dfp["FechaTermino"].isna()
    if mask_need_ini.any() or mask_need_fin.any():
        der = dfp[label_col].apply(_derive_dates_from_label)
        dfp.loc[mask_need_ini, "FechaInicio"] = [t[0] for t in der[mask_need_ini]]
        dfp.loc[mask_need_fin, "FechaTermino"] = [t[1] for t in der[mask_need_fin]]

    # Saca período oficial solicitado
    rowp = dfp.loc[dfp[id_col] == period_id]
    if rowp.empty:
        raise ValueError(f"No encontré el período con Id={period_id} en df_periodos.")
    periodo_label_oficial = str(rowp.iloc[0][label_col])
    inicio_oficial = rowp.iloc[0]["FechaInicio"]
    fin_oficial = rowp.iloc[0]["FechaTermino"]

    # ===== 1) fetch + normalize del SOAP =====
    res = client.service.retornarDiputadosXPeriodo(period_id)
    d = safe_serialize(res) or {}
    df = pd.json_normalize(d, sep=".")

    # ===== 2) Explota militancias =====
    df_explode = df.explode("Diputado.Militancias.Militancia").reset_index(drop=True)

    # ===== 3) Normaliza objeto militancia =====
    df_militancias = (
        json_normalize(df_explode["Diputado.Militancias.Militancia"])
        .rename(columns={
            "FechaInicio": "Militancia.FechaInicio",
            "FechaTermino": "Militancia.FechaTermino",
        })
    )
    
    # ===== 4) Concat + fechas a datetime =====
    df_dip = pd.concat([df_explode, df_militancias], axis=1)
    for col in ["Militancia.FechaInicio", "Militancia.FechaTermino"]:
        if col in df_dip.columns:
            df_dip[col] = pd.to_datetime(df_dip[col], errors="coerce")

    # ===== 5) Mantener SOLO militancias que SOLAPAN con el período oficial =====
    SENTINEL_FUTURO = pd.Timestamp("2099-12-31")
    ini_mil = df_dip["Militancia.FechaInicio"]
    fin_mil = df_dip["Militancia.FechaTermino"].fillna(SENTINEL_FUTURO)
    solapa = (ini_mil <= fin_oficial) & (fin_mil >= inicio_oficial)
    df_dip = df_dip.loc[solapa].copy().set_index("Diputado.Id")

    # --- helper para nombre completo ---
    def _nombre_completo(row):
        partes = [
            row.get("Diputado.Nombre"),
            row.get("Diputado.Nombre2"),
            row.get("Diputado.ApellidoPaterno"),
            row.get("Diputado.ApellidoMaterno"),
        ]
        return " ".join([p for p in partes if p and str(p).strip() != ""]).strip()
    
    # ===== 6) Calcula nombre completo y asigna periodo oficial =====    
    df_dip["nombre_completo"] = df_dip.apply(_nombre_completo, axis=1)
    df_dip["Id_periodo"] = period_id
    df_dip["Periodo"] = periodo_label_oficial

    # ===== 7) militancias_anteriores: inicio < inicio_oficial (cuenta por diputado) =====
    # Ojo: contamos sobre TODO el set (no sólo los que solapan), por eso usamos el df original exploteado
    # pero en caso de querer sólo este subset, podrías usar df_dip en lugar de df_explode+militancias.
    # Aquí lo haremos con todas las militancias disponibles en la respuesta:
    df_all = pd.concat([df_explode, df_militancias], axis=1).set_index("Diputado.Id")
    if "Militancia.FechaInicio" in df_all.columns:
        df_all["Militancia.FechaInicio"] = pd.to_datetime(df_all["Militancia.FechaInicio"], errors="coerce")
    prev_counts = (
        df_all.loc[df_all["Militancia.FechaInicio"] < inicio_oficial]
              .groupby(level=0)
              .size()
              .rename("total_militancias_anteriores")
              .astype(int)
    )
    df_dip["total_militancias_anteriores"] = df_dip.index.map(prev_counts).fillna(0).astype(int)

    # ===== 8) EDAD al inicio oficial del período =====
    # Si no hay fecha oficial (NaT), fallback 11-03 del año inicial del label
    def _ref_date(label, fallback_ini):
        if pd.notna(inicio_oficial):
            return inicio_oficial
        try:
            y_ini = int(str(label).split("-")[0])
            return pd.Timestamp(f"{y_ini}-03-11")
        except Exception:
            return pd.NaT

    df_dip["Diputado.FechaNacimiento"] = pd.to_datetime(df_dip.get("Diputado.FechaNacimiento"), errors="coerce")
    ref_date = _ref_date(periodo_label_oficial, inicio_oficial)
    def _edad_en(dob, ref):
        if pd.isna(dob) or pd.isna(ref):
            return pd.NA
        return int(ref.year - dob.year - ((ref.month, ref.day) < (dob.month, dob.day)))
    df_dip["edad"] = df_dip["Diputado.FechaNacimiento"].apply(lambda dob: _edad_en(dob, ref_date))

    # ===== 9) DROP columnas que ya no interesan =====
    cols_drop = [
        "Distrito.Comunas.Comuna",
        "Diputado.Militancias.Militancia",
        "Diputado.FechaDefucion",
        "Diputado.RUT",
        "Diputado.RUTDV",
        "Partido.Alias",
        "FechaInicio",                 # del tramo 'res' si existiera
        "FechaTermino",                # del tramo 'res' si existiera
        "Diputado.Nombre",
        "Diputado.Nombre2",
        "Diputado.ApellidoPaterno",
        "Diputado.ApellidoMaterno",
        "Diputado.Sexo.Valor"
    ]
    df_dip.drop(columns=[c for c in cols_drop if c in df_dip.columns], errors="ignore", inplace=True)

    # ===== 10) RENOMBRES =====
    rename_map = {
        "Id_periodo": "periodo_id",
        "Diputado.FechaNacimiento": "fecha_nacimiento",
        "Diputado.Sexo._value_1": "sexo",
        "Distrito.Numero": "distrito_numero",
        "Partido.Id": "partido_id",
        "Partido.Nombre": "partido_nombre",
        "total_militancias_anteriores": "militancias_anteriores",
    }
    df_dip = df_dip.rename(columns=rename_map)

    # Asegura 'diputado_id' como columna
    df_dip["diputado_id"] = df_dip.index

    # ===== 11) Tipos útiles =====
    if "fecha_nacimiento" in df_dip.columns:
        df_dip["fecha_nacimiento"] = pd.to_datetime(df_dip["fecha_nacimiento"], errors="coerce")
    for int_col in ["periodo_id", "diputado_id", "distrito_numero", "militancias_anteriores"]:
        if int_col in df_dip.columns:
            if df_dip[int_col].isna().any():
                df_dip[int_col] = pd.to_numeric(df_dip[int_col], errors="coerce").astype("Int64")
            else:
                df_dip[int_col] = pd.to_numeric(df_dip[int_col], errors="coerce").astype(int)

    # ===== 12) Orden final recomendado =====
    ordered_cols = [
        "periodo_id",
        "Periodo",            # label oficial (para carpeta)
        "diputado_id",
        "nombre_completo",
        "fecha_nacimiento",
        "sexo",
        "edad",
        "distrito_numero",
        "partido_id",
        "partido_nombre",
        "militancias_anteriores",
        "Militancia.FechaInicio",
        "Militancia.FechaTermino"
    ]
    for c in ordered_cols:
        if c not in df_dip.columns:
            df_dip[c] = pd.NA

    df_final = df_dip[ordered_cols].reset_index(drop=True)
    return df_final

In [13]:
def sanitize(name):
    """Limpia nombres de carpeta/archivo."""
    if pd.isna(name) or str(name).strip() == "":
        return "sin_periodo"
    return re.sub(r"[^\w\-]+", "_", str(name)).strip("_")

# --- Aseguramos que df_periodos tenga columnas 'Id' y 'Periodo' ---
df_periodos = pd.read_csv(DATA_DIR / 'periodos.csv')
df_periodos_sorted = df_periodos.reset_index().sort_values("Id")

c = Client("https://opendata.camara.cl/camaradiputados/WServices/WSDiputado.asmx?WSDL")

for _, row in df_periodos_sorted.iterrows():
    pid = int(row["Id"])                # ID del período legislativo
    nombre_periodo = str(row["Periodo"])  # Nombre o etiqueta del período, usado para carpeta
    
    if nombre_periodo in periodos:
        try:
            df_pid = estandarizar_df_diputados_x_periodo(c, pid, df_periodos)

            carpeta = DATA_DIR / sanitize(nombre_periodo)   # p.ej., data/1990-1994
            carpeta.mkdir(parents=True, exist_ok=True)

            # ordenar antes de guardar
            order_cols = [c for c in ["nombre_completo", "partido_nombre", "diputado_id"] if c in df_pid.columns]
            df_pid_sorted = df_pid.sort_values(order_cols) if order_cols else df_pid

            out_path = carpeta / "diputados.csv"
            df_pid_sorted.to_csv(out_path, index=False, encoding="utf-8")
            print(f"✓ Guardado {out_path}")

        except Exception as e:
            print(f"✗ Período {pid} ({nombre_periodo}): {e}")

✓ Guardado C:\Users\angel\OneDrive\Documents\U\2025-2\Proyecto de Grado\Legislative-Voting-Behavior-Prediction-\data\2018-2022\diputados.csv
